In [6]:
#Importing Packages
import csv
import urllib2
import os
import pylab as pl
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import statsmodels.api as sm
import statsmodels.formula.api as smf

%pylab inline 

# IMPORTING THE FILE FOR SAT SCORES FOR 2012
SAT_Score = pd.read_csv("https://data.cityofnewyork.us/api/views/f9bf-2cp4/rows.csv?accessType=DOWNLOAD")


Populating the interactive namespace from numpy and matplotlib


In [7]:
SAT_Score.replace('s','NaN', inplace=True, regex=True)
SAT_Score = SAT_Score.dropna()

In [8]:
SAT_Score['District'] = SAT_Score['DBN'].map(lambda x: str(x)[:2])
SAT_Score['District'] = (SAT_Score['District']).astype(float)
# SAT_Score['Num_test_taker'] = (SAT_Score['Num of SAT Test Takers']).astype(float)
SAT_Score['SAT_avg_reading'] = (SAT_Score['SAT Critical Reading Avg. Score']).astype(float)
SAT_Score['SAT_avg_writing'] = (SAT_Score['SAT Writing Avg. Score']).astype(float)
SAT_Score['SAT_avg_math'] = (SAT_Score['SAT Math Avg. Score']).astype(float)

SAT_Score.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478 entries, 0 to 477
Data columns (total 10 columns):
DBN                                478 non-null object
SCHOOL NAME                        478 non-null object
Num of SAT Test Takers             478 non-null object
SAT Critical Reading Avg. Score    478 non-null object
SAT Math Avg. Score                478 non-null object
SAT Writing Avg. Score             478 non-null object
District                           478 non-null float64
SAT_avg_reading                    421 non-null float64
SAT_avg_writing                    421 non-null float64
SAT_avg_math                       421 non-null float64
dtypes: float64(4), object(6)
memory usage: 41.1+ KB


In [9]:
SAT_Score = SAT_Score.groupby('District').mean()

In [10]:
SAT_Score = SAT_Score.reset_index()
SAT_Score = SAT_Score [0:32]

In [11]:
SAT_Score.head(3)

,District,SAT_avg_reading,SAT_avg_writing,SAT_avg_math
0,1,423.777778,413.555556,468.444444
1,2,424.068966,419.448276,441.344828
2,3,424.846154,417.076923,426.000000


In [12]:
SAT_Score['Total_Score'] = pd.DataFrame(SAT_Score['SAT_avg_reading'] + SAT_Score['SAT_avg_writing'] +
                                        SAT_Score['SAT_avg_math'])

In [13]:
SAT_Score.head(3)

,District,SAT_avg_reading,SAT_avg_writing,SAT_avg_math,Total_Score
0,1,423.777778,413.555556,468.444444,1305.777778
1,2,424.068966,419.448276,441.344828,1284.862069
2,3,424.846154,417.076923,426.000000,1267.923077


In [14]:
attendance = pd.read_csv("https://data.cityofnewyork.us/api/views/rfpq-hs49/rows.csv?accessType=DOWNLOAD")
attendance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 0 to 34
Data columns (total 3 columns):
District                  35 non-null object
YTD % Attendance (Avg)    35 non-null object
YTD Enrollment (Avg)      35 non-null int64
dtypes: int64(1), object(2)
memory usage: 1.1+ KB


In [15]:
attendance.head(3)

,District,YTD % Attendance (Avg),YTD Enrollment (Avg)
0,DISTRICT 01,91.18%,12367
1,DISTRICT 02,89.01%,60823
2,DISTRICT 03,89.28%,21962


In [16]:
attendance['District'] = attendance['District'].map(lambda x: str(x)[-2:])

In [17]:
attendance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 0 to 34
Data columns (total 3 columns):
District                  35 non-null object
YTD % Attendance (Avg)    35 non-null object
YTD Enrollment (Avg)      35 non-null int64
dtypes: int64(1), object(2)
memory usage: 1.1+ KB


In [18]:
attendance = attendance [0:32]

In [19]:
attendance['District'] = attendance['District'].astype(float)

In [20]:
type(attendance.iloc[30,1])

str

In [21]:
attendance['class_attend'] = (attendance['YTD % Attendance (Avg)'].map(lambda x: str(x)[:-1])).astype(float)

In [22]:
attendance
type(attendance.iloc[30,3])

numpy.float64

In [23]:
attendance.head()
attendance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 31
Data columns (total 4 columns):
District                  32 non-null float64
YTD % Attendance (Avg)    32 non-null object
YTD Enrollment (Avg)      32 non-null int64
class_attend              32 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.2+ KB


In [24]:
SAT_vs_attend = pd.merge(SAT_Score, attendance, how='inner', on='District')
SAT_vs_attend.columns
SAT_vs_attend.info()
SAT_vs_attend.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 31
Data columns (total 8 columns):
District                  32 non-null float64
SAT_avg_reading           32 non-null float64
SAT_avg_writing           32 non-null float64
SAT_avg_math              32 non-null float64
Total_Score               32 non-null float64
YTD % Attendance (Avg)    32 non-null object
YTD Enrollment (Avg)      32 non-null int64
class_attend              32 non-null float64
dtypes: float64(6), int64(1), object(1)
memory usage: 2.2+ KB


,District,SAT_avg_reading,SAT_avg_writing,SAT_avg_math,Total_Score,YTD % Attendance (Avg),YTD Enrollment (Avg),class_attend
0,1,423.777778,413.555556,468.444444,1305.777778,91.18%,12367,91.18
1,2,424.068966,419.448276,441.344828,1284.862069,89.01%,60823,89.01
2,3,424.846154,417.076923,426.000000,1267.923077,89.28%,21962,89.28


In [25]:
# #Setting X and Y variables
# cls_atnd = SAT_vs_attend['class_attend']
# tot_score = SAT_vs_attend['Total_Score']

# # Fitting linear regression model 
# model_SAT_vs_attend = smf.ols(formula='(tot_score) ~ (cls_atnd) ' , data= SAT_vs_attend).fit()
# print "R-squared value for Attendence vs Total Score:", model_SAT_vs_attend.rsquared
# print model_SAT_vs_attend.params

# # Using limit values for linspace function
# lower1 = np.min(cls_atnd)
# upper1 = np.max(cls_atnd)
# print lower1 , upper1

# # Generating predicted values using the above model1 parameters
# x_dummy = pd.DataFrame({'cls_atnd': np.linspace(lower1, upper1, 100)})
# predicted_model = model_SAT_vs_attend.predict(x_dummy)

# # Fitting a second degree curve linear regression model
# model2_SAT_vs_attend = smf.ols(formula='(tot_score) ~ (cls_atnd) + I(cls_atnd**2)', data= SAT_vs_attend).fit()
# print "R-squared value for Attendence vs Total Score:", model2_SAT_vs_attend.rsquared
# print model2_SAT_vs_attend.params

# # Generating predicted values using the above model2 parameters
# x2_dummy = pd.DataFrame({'cls_atnd': np.linspace(lower1, upper1, 100)})
# predicted2_model = model2_SAT_vs_attend.predict(x_dummy)


In [28]:
class_size = pd.read_csv('https://data.cityofnewyork.us/api/views/urz7-pzb3/rows.csv?accessType=DOWNLOAD')
class_size.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27611 entries, 0 to 27610
Data columns (total 16 columns):
CSD                                     27611 non-null int64
BOROUGH                                 27611 non-null object
SCHOOL CODE                             27611 non-null object
SCHOOL NAME                             27611 non-null object
GRADE                                   26127 non-null object
PROGRAM TYPE                            26127 non-null object
CORE SUBJECT (MS CORE and 9-12 ONLY)    26127 non-null object
CORE COURSE (MS CORE and 9-12 ONLY)     26127 non-null object
SERVICE CATEGORY(K-9* ONLY)             26127 non-null object
NUMBER OF STUDENTS / SEATS FILLED       26127 non-null float64
NUMBER OF SECTIONS                      26127 non-null float64
AVERAGE CLASS SIZE                      26127 non-null float64
SIZE OF SMALLEST CLASS                  26127 non-null float64
SIZE OF LARGEST CLASS                   26127 non-null float64
DATA SOURCE        

In [27]:
classsize_subject = classsize[(classsize.index.get_level_values('GRADE ') == 'MS Core') & 
              ((classsize.index.get_level_values('CORE SUBJECT (MS CORE and 9-12 ONLY)') == 'ENGLISH') |
                (classsize.index.get_level_values('CORE SUBJECT (MS CORE and 9-12 ONLY)') == 'MATH'))]
classsize_subject.head()

NameError: name 'classsize' is not defined

In [ ]:
# pl.figure(figsize=(8, 8))
# pl.title('SAT vs Attendance')
# pl.xlabel('Average School Attendance')
# pl.ylabel('Total SAT Score')
# pl.scatter(cls_atnd,tot_score)
# pl.plot(x_dummy, predicted_model, 'b', label="Fitted Regression Line", linestyle='-')
# pl.plot(x2_dummy,predicted2_model, 'r', linestyle='--', label = "Analytical Regression")
# pl.legend(loc="upper left")
# pl.xlim(int(lower1) , int(upper1+1))
# pl.ylim(1050,1400)


